# Part II.3: Kiểm tra giả định của mô hình AR(1)
## SCENARIO 2: AR(1) trên dữ liệu sau differencing (Extended Analysis)

**Mục tiêu**: Khảo sát xem việc differencing có giúp AR(1) phù hợp hơn không

**Lưu ý**: AR(1) trên differenced data ≡ ARIMA(1,1,0)

In [ ]:
# Import thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Thiết lập style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Đọc dữ liệu
df = pd.read_csv('group1_co2_monthly.csv')
df['index'] = pd.to_datetime(df['index'])
df.set_index('index', inplace=True)
df.index.freq = 'M'

print(f"Số quan sát: {len(df)}")
print(f"Khoảng thời gian: {df.index[0]} đến {df.index[-1]}")

## 0. Exploratory Data Analysis (Tóm tắt)

Từ Scenario 1, ta đã biết:
- Dữ liệu có **trend** tăng mạnh + **seasonal** pattern 12 tháng
- ACF giảm chậm (slow decay) → non-stationary
- AR(1) trên original data FAIL vì φ ≈ 1

**Giải pháp**: Áp dụng **first differencing** để loại bỏ trend

In [ ]:
# Quick visualization
plt.figure(figsize=(14, 5))
plt.plot(df.index, df['co2'], linewidth=1.5, color='blue')
plt.title('Original CO2 Time Series - Có Trend + Seasonality', fontsize=14, fontweight='bold')
plt.xlabel('Time', fontsize=12)
plt.ylabel('CO2 (ppm)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## BƯỚC 1: Kiểm tra Stationarity (Tính dừng)

**Augmented Dickey-Fuller (ADF) test**:
- $H_0$: Chuỗi có unit root (non-stationary)
- $H_1$: Chuỗi stationary
- Reject $H_0$ nếu p-value < 0.05

In [ ]:
# ADF test trên dữ liệu gốc
print("1.1. KIỂM TRA DỮ LIỆU GỐC")
print("="*60)
adf_orig = adfuller(df['co2'], autolag='AIC')

print(f"ADF Statistic: {adf_orig[0]:.4f}")
print(f"P-value: {adf_orig[1]:.6f}")
print(f"Critical Value (5%): {adf_orig[4]['5%']:.4f}")

if adf_orig[1] > 0.05:
    print(f"\n✗ p-value > 0.05 → NON-STATIONARY (có trend)")
else:
    print(f"\n✓ p-value < 0.05 → STATIONARY")

print(f"\nOriginal series - Mean: {df['co2'].mean():.4f}, Std: {df['co2'].std():.4f}")

In [ ]:
# Áp dụng first differencing: ∇Y_t = Y_t - Y_{t-1}
df['co2_diff'] = df['co2'].diff()
df_diff = df.dropna()  # Loại bỏ NaN value đầu tiên

print("\n1.2. KIỂM TRA SAU KHI DIFFERENCING")
print("="*60)
adf_diff = adfuller(df_diff['co2_diff'], autolag='AIC')

print(f"ADF Statistic: {adf_diff[0]:.4f}")
print(f"P-value: {adf_diff[1]:.6f}")
print(f"Critical Value (5%): {adf_diff[4]['5%']:.4f}")

if adf_diff[1] < 0.05:
    print(f"\n✓ p-value < 0.05 → STATIONARY (differencing thành công!)")
else:
    print(f"\n✗ p-value > 0.05 → Vẫn NON-STATIONARY")

print(f"\nDifferenced series - Mean: {df_diff['co2_diff'].mean():.4f}, Std: {df_diff['co2_diff'].std():.4f}")

In [ ]:
# Visualize: So sánh original vs differenced
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Original series
axes[0].plot(df.index, df['co2'], linewidth=1.5, color='blue')
axes[0].set_title('Original CO2 Time Series', fontsize=14, fontweight='bold')
axes[0].set_ylabel('CO2 (ppm)', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].text(0.02, 0.95, f'ADF p-value = {adf_orig[1]:.6f} (Non-stationary)', 
             transform=axes[0].transAxes, fontsize=11, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# Differenced series
axes[1].plot(df_diff.index, df_diff['co2_diff'], linewidth=1.5, color='orange')
axes[1].set_title('First Differenced CO2 Series', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Differenced CO2', fontsize=12)
axes[1].set_xlabel('Time', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].text(0.02, 0.95, f'ADF p-value = {adf_diff[1]:.6f} (Stationary)', 
             transform=axes[1].transAxes, fontsize=11, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

plt.tight_layout()
plt.savefig('outputs/scenario2_stationarity.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Đồ thị đã được lưu: outputs/scenario2_stationarity.png")

### ACF/PACF của differenced data

In [ ]:
# ACF và PACF sau differencing
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

plot_acf(df_diff['co2_diff'], lags=40, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF of Differenced CO2 Data', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=12)
axes[0].set_ylabel('ACF', fontsize=12)
axes[0].grid(True, alpha=0.3)

plot_pacf(df_diff['co2_diff'], lags=40, ax=axes[1], alpha=0.05)
axes[1].set_title('PACF of Differenced CO2 Data', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=12)
axes[1].set_ylabel('PACF', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Quan sát:")
print("  • ACF: Giảm nhanh hơn (so với original) → đã cải thiện stationarity")
print("  • ACF: Vẫn còn peaks tại lag 12, 24, 36 → Seasonality chưa được xử lý")
print("  • PACF: Spike mạnh tại lag 1 → Gợi ý AR(1)")

## BƯỚC 2: Fit AR(1) trên dữ liệu đã differencing

Mô hình: $\nabla Y_t = c + \phi \nabla Y_{t-1} + e_t$

Sử dụng `AutoReg` để fit AR(1)

In [ ]:
# Fit AR(1) model bằng AutoReg
model = AutoReg(df_diff['co2_diff'], lags=1, old_names=False)
fitted_model = model.fit()

print("="*80)
print("MÔ HÌNH AR(1) TRÊN DIFFERENCED DATA - KẾT QUẢ ƯỚC LƯỢNG")
print("="*80)
print(fitted_model.summary())

In [ ]:
# Trích xuất tham số
phi = fitted_model.params['co2_diff.L1']
const = fitted_model.params.get('const', 0)
sigma2 = fitted_model.sigma2

print("\nTHAM SỐ ƯỚC LƯỢNG:")
print(f"  φ̂ (AR coefficient): {phi:.6f}")
print(f"  ĉ (Constant): {const:.6f}")
print(f"  σ̂² (Residual variance): {sigma2:.6f}")

print(f"\n✓ Kiểm tra stationarity: |φ̂| = {abs(phi):.6f} < 1 → Mô hình stationary")

# Lấy residuals
residuals = fitted_model.resid

## BƯỚC 3: Kiểm tra các giả định

### 3.1. Giả định 1: Zero Mean & Constant Variance

In [ ]:
# Tính mean và std
mean_resid = residuals.mean()
std_resid = residuals.std()

print("GIẢ ĐỊNH 1: ZERO MEAN & CONSTANT VARIANCE")
print("="*60)
print(f"Mean của phần dư: {mean_resid:.6f}")
print(f"Std của phần dư: {std_resid:.6f}")
print(f"\n✓ Mean ≈ 0? {abs(mean_resid) < 0.1} (|mean| = {abs(mean_resid):.6f})")
print(f"Status: {'PASS' if abs(mean_resid) < 0.1 else 'FAIL'}")

In [ ]:
# Vẽ biểu đồ
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Residuals plot
axes[0].plot(residuals.index, residuals, linewidth=1, alpha=0.7, color='orange')
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2, label='Zero Line')
axes[0].axhline(y=mean_resid, color='g', linestyle='--', linewidth=2, label=f'Mean = {mean_resid:.4f}')
axes[0].fill_between(residuals.index, -2*std_resid, 2*std_resid, alpha=0.2, color='gray', label='±2σ')
axes[0].set_title('Residuals vs Time (Check: Zero Mean & Constant Variance)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Residuals', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Rolling std
rolling_std = residuals.rolling(window=12).std()
axes[1].plot(rolling_std.index, rolling_std, linewidth=1.5, color='purple', label='Rolling Std (12 months)')
axes[1].axhline(y=std_resid, color='r', linestyle='--', linewidth=2, label=f'Overall Std = {std_resid:.4f}')
axes[1].set_title('Rolling Standard Deviation (Check: Constant Variance)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Std Dev', fontsize=12)
axes[1].set_xlabel('Time', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/scenario2_assumption1.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Đồ thị đã được lưu: outputs/scenario2_assumption1.png")

### 3.2. Giả định 2: Normality

In [ ]:
# Shapiro-Wilk test
shapiro_stat, shapiro_p = stats.shapiro(residuals)

print("GIẢ ĐỊNH 2: NORMALITY TEST")
print("="*60)
print(f"Shapiro-Wilk Test:")
print(f"  Test Statistic: {shapiro_stat:.6f}")
print(f"  P-value: {shapiro_p:.6f}")

if shapiro_p < 0.05:
    print(f"  ✗ p-value < 0.05 → Bác bỏ H₀ (Phần dư KHÔNG có phân phối chuẩn)")
else:
    print(f"  ✓ p-value > 0.05 → Chấp nhận H₀ (Phần dư có phân phối chuẩn)")

print(f"\nStatus: {'PASS' if shapiro_p >= 0.05 else 'FAIL'}")

if shapiro_p < 0.05 and shapiro_p > 0.001:
    print(f"\n⚠️ Lưu ý: Sample size lớn (n={len(residuals)}) → sai lệch nhỏ cũng trở nên significant")

In [ ]:
# Q-Q plot và histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Q-Q plot
stats.probplot(residuals, dist="norm", plot=axes[0])
axes[0].set_title('Q-Q Plot (Check: Normality)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Histogram
axes[1].hist(residuals, bins=30, density=True, alpha=0.7, color='skyblue', edgecolor='black', label='Residuals')
mu, sigma = residuals.mean(), residuals.std()
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[1].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label=f'Normal(μ={mu:.3f}, σ={sigma:.3f})')
axes[1].set_title('Histogram of Residuals with Normal Curve', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Residuals', fontsize=12)
axes[1].set_ylabel('Density', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/scenario2_assumption2.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Đồ thị đã được lưu: outputs/scenario2_assumption2.png")
print("\n📊 Visual check: Nếu các điểm trên Q-Q plot nằm trên đường chéo → phân phối chuẩn")

### 3.3. Giả định 3: No Autocorrelation

In [ ]:
# Ljung-Box test
lb_test = acorr_ljungbox(residuals, lags=20, return_df=True)

print("GIẢ ĐỊNH 3: NO AUTOCORRELATION TEST")
print("="*60)
print("\nLjung-Box Test (10 lags đầu):")
print(lb_test.head(10).to_string())

significant_lags = lb_test[lb_test['lb_pvalue'] < 0.05]
print(f"\nSố lags có p-value < 0.05: {len(significant_lags)}/20")

if len(significant_lags) > 0:
    print(f"Các lags significant: {significant_lags.index.tolist()}")
    print("\n✗ Phát hiện tự tương quan trong residuals")
else:
    print("\n✓ Không phát hiện tự tương quan (Tốt!)")

print(f"\nStatus: {'PASS' if len(significant_lags) == 0 else 'FAIL'}")

In [ ]:
# ACF và PACF plots
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# ACF
plot_acf(residuals, lags=40, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF of Residuals (Check: No Autocorrelation)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=12)
axes[0].set_ylabel('ACF', fontsize=12)
axes[0].grid(True, alpha=0.3)

# PACF
plot_pacf(residuals, lags=40, ax=axes[1], alpha=0.05)
axes[1].set_title('PACF of Residuals', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=12)
axes[1].set_ylabel('PACF', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/scenario2_assumption3.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Đồ thị đã được lưu: outputs/scenario2_assumption3.png")
print("\n📊 Visual check: Nếu ACF values nằm trong confidence bands → không có tự tương quan")

## BƯỚC 4: Tổng kết và so sánh

In [ ]:
print("="*80)
print("TỔNG KẾT - SCENARIO 2: AR(1) ON DIFFERENCED DATA")
print("="*80)

print("\n1. Stationarity:")
print(f"   ADF p-value (original): {adf_orig[1]:.6f}")
print(f"   ADF p-value (differenced): {adf_diff[1]:.6f}")
print(f"   Status: ✓ ACHIEVED (differencing thành công)")

print("\n2. Zero Mean & Constant Variance:")
print(f"   Mean = {mean_resid:.6f}")
print(f"   Status: {'✓ PASS' if abs(mean_resid) < 0.1 else '✗ FAIL'}")

print("\n3. Normality:")
print(f"   Shapiro-Wilk p-value = {shapiro_p:.6f}")
print(f"   Status: {'✓ PASS' if shapiro_p >= 0.05 else '✗ FAIL (minor)'}")

print("\n4. No Autocorrelation:")
print(f"   Significant lags: {len(significant_lags)}/20")
print(f"   Status: {'✓ PASS' if len(significant_lags) == 0 else '✗ FAIL'}")

print("\n" + "="*80)
print("SO SÁNH SCENARIO 1 vs SCENARIO 2")
print("="*80)
print("""
┌─────────────────────────┬───────────────────┬───────────────────┐
│ Assumption              │ Scenario 1 (Orig) │ Scenario 2 (Diff) │
├─────────────────────────┼───────────────────┼───────────────────┤
│ Stationarity            │ ✗ FAIL (φ≈1)      │ ✓ PASS (φ=0.71)   │
│ Zero Mean               │ ✓ PASS            │ ✓ PASS            │
│ Constant Variance       │ ✓ PASS            │ ✓ PASS            │
│ Normality               │ ✗ FAIL (severe)   │ ✗ FAIL (minor)    │
│ No Autocorrelation      │ ✗ FAIL (20/20)    │ ✗ FAIL (20/20)    │
└─────────────────────────┴───────────────────┴───────────────────┘
""")

print("="*80)
print("KẾT LUẬN TỔNG QUÁT")
print("="*80)
print("""
SAU KHI DIFFERENCING:
  ✓ Stationarity: ĐẠT (ADF p < 0.001)
  ✓ Zero mean & Constant variance: ĐẠT
  ✗ Normality: VI PHẠM nhẹ (nhưng chấp nhận được về visual)
  ✗ No autocorrelation: VI PHẠM (vẫn còn seasonality tại lag 12, 24, 36)

CẢI THIỆN SO VỚI SCENARIO 1:
  • Differencing loại bỏ được trend → φ̂ giảm từ 0.999 xuống 0.706
  • Các giả định cơ bản được cải thiện đáng kể
  • NHƯNG AR(1) vẫn KHÔNG THỂ capture seasonal pattern 12 tháng

TẠI SAO VẪN FAIL?
  • AR(1) chỉ dựa vào giá trị lag 1 (tháng trước)
  • Không có cơ chế để capture chu kỳ 12 tháng
  • Residuals vẫn chứa thông tin seasonal chưa được model hóa

💡 KHUYẾN NGHỊ:
  → Sử dụng SARIMA(p,d,q)(P,D,Q)[12] để xử lý cả trend VÀ seasonality
     Ví dụ: SARIMA(1,1,0)(1,0,1)[12]
            - (1,1,0): AR(1) với 1 lần differencing
            - (1,0,1)[12]: Seasonal AR(1) và MA(1) với period=12
""")

print("="*80)